In [3]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/stacking_pkl_file/*.pkl')):
    if 'fm2.pkl' in feat or 'ligbm' in  feat or '2leve' in  feat :
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)


file path ../feature/stacking_pkl_file/Capsule.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_all_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/pseudo_rnn2.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn7879.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_all_shuffle_all.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_pseudo1.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_shuffle1.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_f

In [4]:
for feat in sorted(glob.glob('../feature/other_features/features-vinson/*.csv')):
    if 'fm2.pkl' in feat or 'w2v' in feat or 'lda' in feat or 'lsi' in feat:
        continue
    print('file path',feat)
    file=pd.read_csv(feat)
    a = file.iloc[:102277,:]
    b = file.iloc[102277:,:]
    a = a.values
    b = b.values
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
from catboost import CatBoostRegressor

file path ../feature/other_features/features-vinson/bnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob_0.778620.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/mnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/nn_ensemble_0.775597.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob_0.778881.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/word_fasttext_0.760225.csv
(102277, 19) (102277, 19)
(102277, 399)


In [5]:
import pandas as pd
import numpy as np
train = pd.read_csv("../input/new_data/train_set.csv")
y=(train["class"]-1).astype(int)
del train
import gc
gc.collect()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


236

In [6]:
from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=19)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [10]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
def simple_ens(model_name,k=3,rnd=233,lr=0.05,c_bytree=0.9,s_sample=0.9):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd*3)
    test_pred, val_pred = np.zeros((102277,20)), np.zeros((102277,20))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_f1_l,all_val_f1_l = 0,0
    
    for i in range(19):
        val_loss_l,train_loss_l = 0,0
        val_f1_l,train_f1_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test =test_x
#             params = {
#                 'subsample': s_sample,
#                 'eta': lr,
#                 'max_depth': 10,
#                 'eval_metric':'logloss',
#                 #'eval_metric':'auc',
#                 'objective':'binary:logistic',
#                 #'scale_pos_weight':0.9,
# #                 'colsample_bytree':c_bytree
            
#                 }
            model=CatBoostRegressor(iterations=4000,
                        learning_rate=0.01,
                        depth=3,
                        random_seed=77*i+147,
                        od_type='Iter',
                        od_wait=50)

        
            model.fit(curr_x, curr_y[:,i], eval_set=(hold_out_x, hold_out_y[:,i]), use_best_model=True, verbose=False)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            train_pred = model.predict(curr_x)
            val_pred[test_index, i] = model.predict(hold_out_x)
            curr_train_loss = log_loss(curr_y[:,i],train_pred)
            curr_val_loss = log_loss(hold_out_y[:,i],val_pred[test_index, i])

            curr_train_f1 = self_f1_score(curr_y[:,i],train_pred)
            curr_val_f1 = self_f1_score(hold_out_y[:,i],val_pred[test_index, i])

            print('ls',curr_train_loss,curr_val_loss,'f1',curr_train_f1,curr_val_f1)
            val_loss_l += curr_val_loss
            train_loss_l += curr_train_loss
            val_f1_l += curr_val_f1
            train_f1_l += curr_train_f1
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_f1_l = train_f1_l/k
        val_f1_l = val_f1_l/k
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class f1 train',train_f1_l,'f1 val',val_f1_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/19
        all_val_loss_l += val_loss_l/19
        all_train_f1_l += train_f1_l/19
        all_val_f1_l += val_f1_l/19
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all f1 avg',all_train_f1_l,all_val_f1_l)
    print('=======================================================')
    return val_pred, test_pred

print('done')

done


In [11]:
xgb_res = simple_ens('xgb',10,233,0.05,0.8,0.7)

0 fold: ls 0.09448820318714928 0.10171127253846005 f1 0.8044604282756151 0.8105114310766659
1 fold: ls 0.09569346813157752 0.10173917997101449 f1 0.80114437608762 0.7933058464221099
2 fold: ls 0.09362486809126339 0.1045140409209866 f1 0.8062738154043512 0.7788538972823376
3 fold: ls 0.09291273294131164 0.10219493599854317 f1 0.8079657686513295 0.7928969685544629
4 fold: ls 0.0949444810341598 0.0971365201614851 f1 0.8063458542141709 0.7923610446265129
5 fold: ls 0.09448947826050025 0.09612727309569168 f1 0.8040695557573301 0.8116423691400692
6 fold: ls 0.09357791242190752 0.10067214646189117 f1 0.8060724603385865 0.795313958889738
7 fold: ls 0.09493433753520758 0.09330245653402443 f1 0.8036646565026468 0.788396986675238
8 fold: ls 0.09384438815006832 0.09594914082368534 f1 0.8056270279513014 0.7952425439361268
9 fold: ls 0.09748047417762766 0.09527891869446259 f1 0.7979969743837438 0.8044519416438918
this class avg train 0.09459903439307729 avg val 0.09886258852002447
this class f1 trai

6 fold: ls 0.07873438769985705 0.07757865636409952 f1 0.8724472990083342 0.8692035806247047
7 fold: ls 0.07876593438499446 0.08111981537456893 f1 0.8722545388938432 0.8690154098664903
8 fold: ls 0.07951799494725079 0.07779890676333277 f1 0.871468729438223 0.8721266129526674
9 fold: ls 0.07802795145030193 0.08398285134584799 f1 0.87352983169274 0.8713525192259812
this class avg train 0.07785926203783529 avg val 0.08061544756353367
this class f1 train 0.8739262745227105 f1 val 0.8700183970977818
0 fold: ls 0.023691282842181877 0.03086497106866742 f1 0.9735607593911608 0.9667845254321088
1 fold: ls 0.0256064292035961 0.02555701410317879 f1 0.9716459712793277 0.9705762299093661
2 fold: ls 0.024889313055167363 0.025378832102257644 f1 0.972353152065697 0.9719426039248787
3 fold: ls 0.02441329749185907 0.02886635916745786 f1 0.9724366930144561 0.9665674743172187
4 fold: ls 0.024902297628971577 0.0270551849914078 f1 0.9722733631827697 0.9705037631005136
5 fold: ls 0.02495857610552816 0.0235985

2 fold: ls 0.02384427491260517 0.028834759853917525 f1 0.9248167816342563 0.9107470378056597
3 fold: ls 0.023844457499505423 0.02676486605099058 f1 0.9239882542306077 0.9182419670528692
4 fold: ls 0.023109382039975573 0.028609724300521788 f1 0.9270869469285825 0.912662745154903
5 fold: ls 0.025111369790402117 0.027938812184470408 f1 0.9207809136262577 0.9242006521160382
6 fold: ls 0.02455808344580824 0.03385619635548461 f1 0.9235696562214502 0.909244878423128
7 fold: ls 0.02375399946513281 0.033485121313922096 f1 0.9244588055570789 0.9206912235087177
8 fold: ls 0.024439886500606783 0.030552548919210422 f1 0.9229460676978704 0.9111524447119254
9 fold: ls 0.0244496770343862 0.029279505775192498 f1 0.9223476252373337 0.9180496544112356
this class avg train 0.024382594245564132 avg val 0.029021277683645458
this class f1 train 0.9231688708474437 f1 val 0.9180667447024653
0 fold: ls 0.03450773581576632 0.035221905333638934 f1 0.8887425706348859 0.8850458754933508
1 fold: ls 0.035593533448465

In [14]:
xgb_res[1][0]

array([1.27574167e-03, 5.06770693e-04, 1.44507639e-03, 9.48748137e-04,
       9.85466886e-01, 1.27515352e-03, 1.14046840e-03, 1.88301657e-03,
       2.37178589e-04, 1.95817456e-03, 8.96503254e-04, 1.92631510e-03,
       1.66390463e-03, 7.47787762e-04, 4.24425648e-04, 9.93189845e-04,
       8.84350626e-04, 1.15364068e-03, 2.61866658e-03, 0.00000000e+00])

In [15]:
with open('../feature/2leve_cat_stacking.pkl','wb') as fout:
    pickle.dump([xgb_res[0],xgb_res[1]],fout)

In [16]:
test_dir = '../input/new_data/test_set.csv'
test = pd.read_csv(test_dir)
test_id = test[["id"]].copy()

In [18]:
preds=np.argmax(xgb_res[1],axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../output/catstacknnx1.csv',index=None)

(102277, 1)
(102277, 1)


In [19]:
name = ["class_prob_%s"%i for i in range(1,21)]
df_lgb = pd.DataFrame(xgb_res[1], columns=name)
df_lgb = df_lgb.drop('class_prob_20', axis=1)

In [21]:

df_lgb.to_csv('../pro/stacking_cat.csv', index=False)

In [22]:
df_lgb

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,0.001276,0.000507,0.001445,0.000949,0.985467,0.001275,0.001140,0.001883,0.000237,0.001958,0.000897,0.001926,0.001664,0.000748,0.000424,0.000993,0.000884,0.001154,0.002619
1,0.004931,0.000310,0.000798,0.984282,0.000494,0.000158,0.000914,0.000287,-0.000204,0.000934,0.000605,0.001622,0.000244,0.000884,-0.000271,0.000283,0.000283,0.001780,0.001297
2,0.046735,0.003574,0.060400,0.003073,0.008393,0.004201,0.006411,0.062449,0.004582,0.006368,0.010612,0.041658,0.718284,0.004775,0.005102,0.000812,0.004227,0.004723,0.019668
3,0.001337,0.000120,0.000421,0.989435,0.000342,0.000021,0.000671,0.000024,-0.000301,0.000817,0.000274,0.001055,0.000081,0.000727,-0.000335,0.000248,0.000221,-0.000236,0.000837
4,0.013278,0.001127,0.001989,0.001449,0.950787,0.001575,0.003392,0.002971,0.000876,0.002856,0.001298,0.002027,0.002528,0.008082,0.000758,0.001060,0.001552,0.002857,0.004327
5,0.022727,0.002396,0.002799,0.001580,0.929502,0.003229,0.003042,0.006395,0.001121,0.006418,0.007079,0.004113,0.005310,0.002042,0.001640,0.001027,0.002298,0.002701,0.006025
6,0.013815,0.002103,0.002962,0.001623,0.001459,0.001688,0.003293,0.005714,0.001906,0.040748,0.003459,0.004938,0.004772,0.002471,0.915870,0.000513,0.002576,0.005723,0.019901
7,0.029823,0.142414,0.003103,0.002841,0.003311,0.005805,0.056334,0.010454,0.003447,0.009321,0.005689,0.008990,0.018299,0.004370,0.008564,0.000858,0.062049,0.020375,0.593311
8,0.000586,0.000086,0.996627,0.000049,0.000332,0.001362,0.000471,-0.000301,-0.000072,0.000546,0.000315,0.000875,-0.000129,-0.000209,-0.000339,0.000083,-0.000062,-0.000367,0.000604
9,0.116277,0.003430,0.003407,0.002173,0.003067,0.002203,0.015660,0.157291,0.012225,0.149667,0.006596,0.502219,0.025561,0.003137,0.003056,0.000282,0.003419,0.014587,0.058232
